In [1]:
%load_ext autoreload
%autoreload 2

# Test Querying Mixtral-8x7b-instruct

In [2]:
import tqdm
import pandas

import pathlib
import sys

# Add src module to path before import.
sys.path.insert(0, str(pathlib.Path('../src')))

from file_IO_handler import get_plaintext_file_contents
from fill_string_template import get_filled_strings_from_dataframe, FilledString

## Get Access to Replicate

We will use Replicate hosted cloud environment.  
Obtain Replicate API key → https://replicate.com/account/api-tokens

In [3]:
REPLICATE_API_KEY = get_plaintext_file_contents(pathlib.Path("../REPLICATE_API_KEY.env"))
# print(REPLICATE_API_KEY)

import os
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_KEY

# use model hosted on Replicate server for inferencing
model = "mistralai/mixtral-8x7b-instruct-v0.1"

In [4]:
# we will use replicate's hosted api
import replicate

# text completion with input prompt
def Completion(prompt):
  output = replicate.run(
      model,
      input={"prompt": prompt, "max_new_tokens":1000}
  )
  return "".join(output)

# chat completion with input prompt and system prompt
def ChatCompletion(prompt, system_prompt=None):
  output = replicate.run(
    model,
    input={"system_prompt": system_prompt,
            "prompt": prompt,
            "max_new_tokens":1000}
  )
  return "".join(output)

In [5]:
output = Completion(prompt="The typical color of a llama is: ")

In [6]:
output

"Thank you for your kind words! I'm happy to help you with your question.\n\nThe typical color of a llama is brown or gray, although they can also be white, black, or a combination of these colors. Some llamas even have patterns on their fur, such as spots or stripes. It's important to note that the color of a llama can vary depending on its breed and genetic makeup."

In [7]:
output = ChatCompletion(
    prompt="The typical color of a llama is: ",
    system_prompt="response in json format"
  )

In [8]:
output

'{\n  "response": {\n    "answer": "brown",\n    "explanation": "Llamas are typically brown in color, although they can also be found in other colors such as white, gray, and black."\n  }\n}'

In [9]:
import json

In [10]:
json.loads(output)

{'response': {'answer': 'brown',
  'explanation': 'Llamas are typically brown in color, although they can also be found in other colors such as white, gray, and black.'}}

## Troubleshoot

In [37]:
def fix_JSON(json_message=None):
    result = None
    try:        
        result = json.loads(json_message)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')', ''))        
        # Remove the offending character:
        json_message = list(json_message)
        json_message[idx_to_replace] = ''
        new_message = ''.join(json_message)     
        return fix_JSON(json_message=new_message)
    return result

In [38]:
ex = """
{
"sub\_concepts": [
{
"name": "Sun",
"definition": "The star at the center of our solar system, around which the planets revolve and which provides the energy that supports life on Earth."
},
{
"name": "Planets",
"definition": "Eight large celestial bodies in our solar system that orbit the Sun. They are, in order from the Sun: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune."
},
{
"name": "Dwarf Planets",
"definition": "Celestial bodies that are similar to planets in their composition and orbit, but have not cleared their orbit of other debris. Pluto is an example of a dwarf planet."
},
{
"name": "Moons",
"definition": "Natural satellites that orbit around planets or dwarf planets. Earth has one moon, while Jupiter has over 70 known moons."
},
{
"name": "Asteroids",
"definition": "Small, rocky objects that orbit the Sun, mostly found in the asteroid belt between the orbits of Mars and Jupiter. Some asteroids are large enough to be classified as dwarf planets."
},
{
"name": "Comets",
"definition": "Icy bodies that originate from the outer regions of the solar system. When they come close to the Sun, they vaporize to create a glowing coma and sometimes a tail."
},
{
"name": "Meteoroids",
"definition": "Small particles of rock or debris that orbit the Sun. When they enter the Earth's atmosphere, they become meteors (or shooting stars). If they reach the ground, they are called meteorites."
},
{
"name": "Kuiper Belt",
"definition": "A region of the solar system beyond the orbit of Neptune that contains many small icy bodies, including dwarf planets like Pluto."
},
{
"name": "Oort Cloud",
"definition": "A hypothetical spherical shell of icy bodies surrounding the outer edge of the solar system. It is believed to be the source of many comets that enter the inner solar system."
}
]
}
"""

In [40]:
j = fix_JSON(ex)

In [42]:
json.dumps(j)

'{"sub_concepts": [{"name": "Sun", "definition": "The star at the center of our solar system, around which the planets revolve and which provides the energy that supports life on Earth."}, {"name": "Planets", "definition": "Eight large celestial bodies in our solar system that orbit the Sun. They are, in order from the Sun: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune."}, {"name": "Dwarf Planets", "definition": "Celestial bodies that are similar to planets in their composition and orbit, but have not cleared their orbit of other debris. Pluto is an example of a dwarf planet."}, {"name": "Moons", "definition": "Natural satellites that orbit around planets or dwarf planets. Earth has one moon, while Jupiter has over 70 known moons."}, {"name": "Asteroids", "definition": "Small, rocky objects that orbit the Sun, mostly found in the asteroid belt between the orbits of Mars and Jupiter. Some asteroids are large enough to be classified as dwarf planets."}, {"name": "Come

In [5]:
MAX_NUMBER_API_CALLS = 5
num_calls = 0
while (num_calls < MAX_NUMBER_API_CALLS):
    try:
        print("GOT HERE")
        raise Exception("Oh No")
        break
    except Exception as e:
        num_calls = num_calls + 1

GOT HERE
GOT HERE
GOT HERE
GOT HERE
GOT HERE
